In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy as sp
import h5py

import holodeck as holo
import holodeck.single_sources as ss
from holodeck.constants import YR, MSOL
from holodeck import utils, detstats, plot


import hasasia.sensitivity as hsen
import hasasia.sim as hsim

# Read in hdf file

In [ ]:
filepath = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss64_09Bn1000_2023-06-22_uniform-09b_n1000_r100_f40_l10'
filename = filepath+'/sam_lib.hdf5'
ssfile = h5py.File(filename, 'r')
print(list(ssfile.keys()))
fobs = ssfile['fobs'][:]
dur = 1/fobs[0]
cad = 1/(2*fobs[-1])
hc_ss_all = ssfile['hc_ss'][...]
hc_bg_all = ssfile['hc_bg'][...]
shape = hc_ss_all.shape
nsamp, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
print('N,F,R,L:', nsamp, nfreqs, nreals, nloudest)
ssfile.close()

In [ ]:
# find best
# found this in a another notebook to be sample 30
# nn=30
fidx=1 # choose second bin
hc_ref15_10yr = 11.2*10**-15 
print(fobs[fidx]*YR)
nsort, fidx, hc_ref15 = detstats.rank_samples(hc_ss_all, hc_bg_all, fobs, fidx=1, 
                              hc_ref=hc_ref15_10yr, ret_all=True)
nn = nsort[0]
print(nn, fobs[fidx]*YR, hc_ref15)

In [ ]:
dp_bg = detstats.detect_bg_pta

In [ ]:

def get_dpbg(hc_bg, npsrs, sigma, trials):
    avg_dp_tot = 0
    # median_tot = 0
    for ii in range(trials):
        # build PTA
        phis = np.random.uniform(0, 2*np.pi, size = npsrs)
        thetas = np.random.uniform(np.pi/2, np.pi/2, size = npsrs)
        # sigmas = np.ones_like(phis)*sigma
        psrs = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=sigma,
                        phi=phis, theta=thetas)
        dp_bg = detstats.detect_bg_pta(psrs, fobs, hc_bg=hc_bg)
        # print(f'{dp_bg=}')
        avg_dp_tot+=np.median(dp_bg)
    avg_dp = avg_dp_tot/trials
        # median_tot+=np.median(dp_bg) #, np.std(dp_bg))
    return avg_dp

In [ ]:
NREALS = nreals
NPSRS = 40
SIG_START = 1e-7
SIG_MIN = 1e-9
SIG_MAX = 1e-3

In [ ]:
# def get_dpbg(hc_bg, npsrs, sigma, trials, fobs, dur, cad, nreals=NREALS):
#     all_dp = np.zeros((trials, nreals))
#     # median_tot = 0
#     for ii in range(trials):
#         # build PTA
#         phis = np.random.uniform(0, 2*np.pi, size = npsrs)
#         thetas = np.random.uniform(np.pi/2, np.pi/2, size = npsrs)
#         # sigmas = np.ones_like(phis)*sigma
#         psrs = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=sigma,
#                         phi=phis, theta=thetas)
#         all_dp[ii] = detstats.detect_bg_pta(psrs, fobs, hc_bg=hc_bg)
#         # print(f'{dp_bg=}')
#     avg_dp = np.mean(all_dp)
#     std_dp = np.std(all_dp)
#         # median_tot+=np.median(dp_bg) #, np.std(dp_bg))
#     return avg_dp, std_dp

# def quick_pta_calibration(hc_bg, fobs, maxtrials=2, debug=False,
#                           sig_start = 1e-6, sig_min = 1e-10, sig_max = 1e-3, npsrs=NPSRS):
#     assert maxtrials>1, "'maxtrials' must be >1"
#     dur = 1.0/fobs[0]
#     cad = 1.0/(2*fobs[-1])
#     sigma=sig_start

#     avg_dp = 0
#     trials=1
#     while avg_dp<.495 or avg_dp>.505 or trials==1:
#         avg_dp, std_dp = get_dpbg(hc_bg, npsrs=npsrs, sigma=sigma, trials=trials,
#                           fobs=fobs, dur=dur, cad=cad)
#         if debug: print(f"{avg_dp=}, {sigma=}, {sig_min=}, {sig_max=}, {trials=}")
        
#         if avg_dp>0.4 and avg_dp<0.5:
#             trials=maxtrials

#         if avg_dp>=0.51:  # avg_dp too high, raise sigma
#             sig_min = sigma
#             sigma = np.random.uniform(sig_min, sig_max)
#         elif avg_dp<=0.49: # avg_dp too low, decrease sigma
#             sig_max = sigma
#             sigma = np.random.uniform(sig_min, sig_max)     
#     return sigma, avg_dp, std_dp

In [ ]:
def get_dpbg(hc_bg, npsrs, sigma, trials, fobs, dur, cad, nreals=NREALS):
    all_dp = np.zeros((trials, nreals))
    # median_tot = 0
    for ii in range(trials):
        # build PTA
        phis = np.random.uniform(0, 2*np.pi, size = npsrs)
        thetas = np.random.uniform(np.pi/2, np.pi/2, size = npsrs)
        # sigmas = np.ones_like(phis)*sigma
        psrs = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=sigma,
                        phi=phis, theta=thetas)
        all_dp[ii] = detstats.detect_bg_pta(psrs, fobs, hc_bg=hc_bg)
        # print(f'{dp_bg=}')
    avg_dp = np.mean(all_dp)
    std_dp = np.std(all_dp)
        # median_tot+=np.median(dp_bg) #, np.std(dp_bg))
    return avg_dp, std_dp

def quick_pta_calibration(hc_bg, fobs, maxtrials=2, debug=False,
                          sig_start = 1e-6, sig_min = 1e-10, sig_max = 1e-3, npsrs=NPSRS,):
    dur = 1.0/fobs[0]
    cad = 1.0/(2*fobs[-1])
    sigma=sig_start

    avg_dp = 0
    trials = 1
    enough_trials = False
    while avg_dp<.495 or avg_dp>.505 or enough_trials==False: # must be within the desired target range using the max number of trials
        avg_dp, std_dp = get_dpbg(hc_bg, npsrs=npsrs, sigma=sigma, trials=trials,
                          fobs=fobs, dur=dur, cad=cad)
        if debug: print(f"{avg_dp=}, {sigma=}, {sig_min=}, {sig_max=}, {trials=}")
        
        if avg_dp>0.4 and avg_dp<0.5:
            if trials == maxtrials:
                enough_trials=True
            else:
                trials=maxtrials

        if avg_dp>=0.51:  # avg_dp too high, raise sigma
            sig_min = sigma
            sigma = np.random.uniform(sig_min, sig_max)
        elif avg_dp<=0.49: # avg_dp too low, decrease sigma
            sig_max = sigma
            sigma = np.random.uniform(sig_min, sig_max)     
    return sigma, avg_dp, std_dp

def binary_pta_calibration(hc_bg, fobs, maxtrials=2, debug=False,
                          sig_start = 1e-6, sig_min = 1e-10, sig_max = 1e-3, npsrs=NPSRS,):
    dur = 1.0/fobs[0]
    cad = 1.0/(2*fobs[-1])
    sigma=sig_start

    avg_dp = 0
    trials = 1
    enough_trials = False
    while avg_dp<.495 or avg_dp>.505 or enough_trials==False: # must be within the desired target range using the max number of trials
        avg_dp, std_dp = get_dpbg(hc_bg, npsrs=npsrs, sigma=sigma, trials=trials,
                          fobs=fobs, dur=dur, cad=cad)
        if debug: print(f"{avg_dp=}, {sigma=}, {sig_min=}, {sig_max=}, {trials=}")
        
        if avg_dp>0.4 and avg_dp<0.5:
            if trials == maxtrials:
                enough_trials=True
            else:
                trials=maxtrials

        if avg_dp>=0.51:  # avg_dp too high, raise sigma
            sig_min = sigma
            sigma = np.mean([sig_min, sig_max])
        elif avg_dp<=0.49: # avg_dp too low, decrease sigma
            sig_max = sigma
            sigma = np.mean([sig_min, sig_max])     
    return sigma, avg_dp, std_dp

In [ ]:
def logmean(arr1, axis=None):
    """ Get the geometric (log space) mean over two arrays """
    log1 = np.log10(arr1)
    logmean = np.mean(log1, axis=axis)
    mean = 10**logmean
    return mean

print('%.2e' % logmean([0.001, 0.01]))


In [ ]:
print(f"{nreals=}")
sigma, avg_dp, std_dp = quick_pta_calibration(hc_bg_all[nn], fobs, maxtrials=1)
print(f"{sigma=:.3e}, {avg_dp=:.3f}, {std_dp=:.3f}")
%timeit sigma, avg_dp, std_dp = quick_pta_calibration(hc_bg_all[nn], fobs, maxtrials=1)


In [ ]:
print(f"{nreals=}")
sigma, avg_dp, std_dp = binary_pta_calibration(hc_bg_all[nn], fobs, maxtrials=1)
print(f"{sigma=:.3e}, {avg_dp=:.3f}, {std_dp=:.3f}")
%timeit sigma, avg_dp, std_dp = binary_pta_calibration(hc_bg_all[nn], fobs, maxtrials=1)

In [ ]:
print(f"{nreals=}")
sigma, avg_dp, std_dp = quick_pta_calibration(hc_bg_all[nn], fobs, maxtrials=2)
print(f"{sigma=:.3e}, {avg_dp=:.2f}, {std_dp=:.3f}")
%timeit sigma, avg_dp, std_dp = quick_pta_calibration(hc_bg_all[nn], fobs, maxtrials=2)

In [ ]:
print(f"{nreals=}")
sigma, avg_dp, std_dp = binary_pta_calibration(hc_bg_all[nn], fobs, maxtrials=2)
print(f"{sigma=:.3e}, {avg_dp=:.3f}, {std_dp=:.3f}")
%timeit sigma, avg_dp, std_dp = binary_pta_calibration(hc_bg_all[nn], fobs, maxtrials=2)

# Function from detstats.py

In [ ]:
print(f"{nreals=}")
sigma, avg_dp, std_dp = detstats.binary_pta_calibration(hc_bg_all[nn], fobs, NPSRS, maxtrials=1)
print(f"{sigma=:.3e}, {avg_dp=:.3f}, {std_dp=:.3f}")
%timeit sigma, avg_dp, std_dp = detstats.binary_pta_calibration(hc_bg_all[nn], fobs, NPSRS, maxtrials=1)

In [ ]:
print(f"{nreals=}")
sigma, avg_dp, std_dp = detstats.binary_pta_calibration(hc_bg_all[nn], fobs, NPSRS, maxtrials=5)
print(f"{sigma=:.3e}, {avg_dp=:.3f}, {std_dp=:.3f}")
%timeit sigma, avg_dp, std_dp = detstats.binary_pta_calibration(hc_bg_all[nn], fobs, NPSRS, maxtrials=5)

# Individual Realizations

In [ ]:
hc_bg = hc_bg_all[nn]

import time

def calibrate_every_real(hc_bg, fobs, npsrs, maxtrials):
    nreals = hc_bg.shape[-1]
    rsigmas = np.zeros(nreals)
    avg_dps = np.zeros(nreals)
    std_dps = np.zeros(nreals)
    for rr in range(nreals):
        hc_bg_rr = hc_bg[:,rr:rr+1]
        # print(hc_bg_rr.shape)
        rsigmas[rr], avg_dps[rr], std_dps[rr] = detstats.binary_pta_calibration(
            hc_bg_rr, fobs, npsrs, maxtrials, sig_min=1e-8, sig_max=1e-4
        )
    return rsigmas, avg_dps, std_dps
start_time = time.time()
rsigmas, avg_dps, std_dps = calibrate_every_real(hc_bg_all[nn], fobs, NPSRS, 2)
print(f"{time.time() - start_time} s")

In [ ]:
%time rsigmas, avg_dps, std_dps = calibrate_every_real(hc_bg_all[nn], fobs, NPSRS, 1)
%time rsigmas, avg_dps, std_dps = calibrate_every_real(hc_bg_all[nn], fobs, NPSRS, 2)
%time rsigmas, avg_dps, std_dps = calibrate_every_real(hc_bg_all[nn], fobs, NPSRS, 3)


In [ ]:
%time rsigmas, avg_dps, std_dps = detstats.calibrate_every_real(hc_bg_all[nn], fobs, NPSRS, 4)

In [ ]:
print(hc_ss_all.shape)

In [ ]:
hc_ss = hc_ss_all[nn]
print(hc_ss.shape)
hc_bg = hc_bg_all[nn]
NSKIES=20

%time detstats.detect_pspace_model_clbrt_pta(fobs, hc_ss, hc_bg, NPSRS, nskies=20, debug=True, tol=0.01, maxbads=10)

In [ ]:
%time detstats.detect_pspace_model_clbrt_pta(fobs, hc_ss, hc_bg, NPSRS, NSKIES, debug=True, tol=0.01, maxbads=20)

In [ ]:
%time detstats.detect_pspace_model_clbrt_pta(fobs, hc_ss, hc_bg, NPSRS, NSKIES, debug=True, tol=0.01, maxbads=30)